<a href="https://colab.research.google.com/github/cric96/DL-exercise/blob/main/test_with_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Network applied in Aggregate Computing
In this notebook, I tried to apply Recurrent Neural Networks (RNN) in the context of Aggregate Computing (AC).

## Model

Usually, RNN are trained in indipendent sequences. In case of Aggregate Computing, the temporal sequence are correlated with each other following a neighbourhood policy. 

The key idea here is:
- express the system as a graph. Offline we can imagine to access to the entire system
- in each time step, we aggregate data from the neighbours and then we use RNN to compute the right output

# Imports

In [86]:
import tensorflow as tf
import tensorboard
from datetime import datetime
from numpy.random import seed
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Simple Graph

In this model, I model graph as:
$G = (N, E)$

Where, $N$ contains a feature vector and an output vector.

$E$ is expressed as adjacency matrix, so: $E_{i,j} = 1$ iff $i$ is neighbour of $j$.

Input is a matrix contains the concatenation of feature vector and output vector, something like: $ I(n) = <f(n), o(n)>$.
Where $<a, b, c>$ means the column-wise fector concatenation. 
For instance, giving a vector $a = [1, 2]$ and $b = [2, 3]$, $<a,b> = [1, 2, 2, 3]$.

The neighbour aggregation data can be computed leveraging matrix multiplication --- so increasing traning performance.
$$ reduction(E * F) $$
Where F contains all node features.
$reduction$ is a multiset function (e.g. summation, mean,...).

Pay attention, this model can be easly used in a decentralized situation.
Indeed, the data aggregation will be done without global adjency matrix but only retrieving neighbour data. The dense layer already works locally.

In [154]:
feature = tf.constant([
                     [1], 
                     [0],
                     [0],
                     [0],
                    ], dtype=tf.float32)
output = tf.constant([
                     [-1.0], 
                     [-1.0],
                     [-1.0],
                     [-1.0],
                    ], dtype=tf.float32)

input = tf.concat([feature, output], axis = 1)
input = input[:,tf.newaxis, :]
neigh = tf.constant([
                     [0, 1, 0, 0], 
                     [1, 0, 1, 0],
                     [0, 1, 0, 1],
                     [0, 0, 1, 0],
                    ], dtype=tf.float32)

ground = tf.constant([
                     [1, 0], 
                     [0, 1],
                     [0, 2],
                     [0, 3],
                    ], dtype=tf.float32)

# Forward Pass

Taking the output of previous step, adjecency matrix, and feature vector, this function compute the value of the next evaluation.
So
1. compute neighbourhood feature via aggregation
2. concat neighbourhood feature with local feature and previous output
3. perform a forward pass of a neural network.


In [171]:
def forward(result, neigh, feature, eval_logic, log_enable=False):
  if(log_enable):
    print("Input = ", result)
  reshape = tf.reshape(result[:,:, 1], feature.shape[0])
  neigh_evaluation = tf.reduce_sum(tf.multiply(neigh, reshape), 1)
  input_network = tf.concat([result[:,:, 0], neigh_evaluation[:, tf.newaxis]], 1)
  if(log_enable):
    print("Neighbour Aggregation = ", input_network)
  result = eval_logic(input_network[:, tf.newaxis])
  other = tf.concat([feature, result], axis = 1)
  if(log_enable):
    print("Output = ", result)
  return tf.expand_dims(other, [1])

# Model creation
The forward pass needs a function to compute the next output. In this case I use RNN combined with a Dense layer (i.e. a matrix multiplicaiton)

In [178]:
def create_model(input_shape):
  lstm = tf.keras.layers.LSTM(units = 10, activation='relu', return_sequences=False, stateful=True)
  dense = tf.keras.layers.Dense(units = 1, activation='relu')
  input_layer = tf.keras.layers.InputLayer(input_shape=input.shape[1:], batch_size=input_shape[0])
  return tf.keras.Sequential(
      [input_layer, lstm, dense]
  )
model = create_model(input.shape)

# Train function

In [182]:
## TODO pass input and network
def train(model, data, iteration, stabilise_in, stabilisation_check, loss, optimizer, each=100):
  input, neigh, feature, ground = data
  for j in range(iteration):
    with tf.GradientTape() as tape:
      result = input
      to_backprop = 0
      for i in range(stabilise_in):
        result = forward(result, neigh, feature, model)
      for i in range(stabilisation_check):
        result = forward(result, neigh, feature, model)
        to_backprop += 1 / stabilisation_check * loss(ground, result[:, 0, :])
      model.reset_states()
      gradient = tape.gradient(to_backprop, model.weights)
      optimizer.apply_gradients(zip(gradient, model.weights))
    if(j % each == 0):
      print("Epoch ", j ,"Loss = ", tf.reduce_sum(to_backprop))  


## RNN With Aggregation


In [186]:
#seed(42)
#tf.random.set_seed(42)
model = create_model(input.shape) ## comment to avoid the recomputation of weights
iteration = 5000
stabilise_in = 3
stabilisation_check = 5

loss = tf.losses.mse
optimizer = tf.optimizers.Adam()
data = (input, neigh, feature, ground)
train(model, data, iteration, stabilise_in, stabilisation_check, loss, optimizer)

model.reset_states()
result = input
for i in range(20):
  result = forward(result, neigh, feature, model, True)
print(result)
model.reset_states()

Epoch  0 Loss =  tf.Tensor(6.990353, shape=(), dtype=float32)
Epoch  100 Loss =  tf.Tensor(3.1402464, shape=(), dtype=float32)
Epoch  200 Loss =  tf.Tensor(1.7155447, shape=(), dtype=float32)
Epoch  300 Loss =  tf.Tensor(0.5925666, shape=(), dtype=float32)
Epoch  400 Loss =  tf.Tensor(0.2531045, shape=(), dtype=float32)
Epoch  500 Loss =  tf.Tensor(0.07508902, shape=(), dtype=float32)
Epoch  600 Loss =  tf.Tensor(0.031292252, shape=(), dtype=float32)
Epoch  700 Loss =  tf.Tensor(0.022025175, shape=(), dtype=float32)
Epoch  800 Loss =  tf.Tensor(0.008005813, shape=(), dtype=float32)
Epoch  900 Loss =  tf.Tensor(0.004392569, shape=(), dtype=float32)
Epoch  1000 Loss =  tf.Tensor(0.0033386755, shape=(), dtype=float32)
Epoch  1100 Loss =  tf.Tensor(0.0024593782, shape=(), dtype=float32)
Epoch  1200 Loss =  tf.Tensor(0.0016910656, shape=(), dtype=float32)
Epoch  1300 Loss =  tf.Tensor(0.0011480821, shape=(), dtype=float32)
Epoch  1400 Loss =  tf.Tensor(0.0008091271, shape=(), dtype=float32)

In [168]:
feature_validation = tf.constant([
                     [1], 
                     [0],
                     [0],
                     [0],
                     [0]
                    ], dtype=tf.float32)
output_validation = tf.constant([
                     [-1.0], 
                     [-1.0],
                     [-1.0],
                     [-1.0],
                     [-1.0]
                    ], dtype=tf.float32)

input_validation = tf.concat([feature_validation, output_validation], axis = 1)
input_validation = input_validation[:,tf.newaxis, :]
neigh_validation = tf.constant([
                     [0, 1, 0, 0, 0], 
                     [1, 0, 1, 0, 0],
                     [0, 1, 0, 1, 0],
                     [0, 0, 1, 0, 1],
                     [0, 0, 0, 1, 0],
                    ], dtype=tf.float32)

change_model = create_model(input_validation.shape)
change_model.set_weights(model.get_weights())

result_validation = input_validation
model.reset_states()
for i in range(20):
  result = forward(result_validation, neigh_validation, feature_validation, change_model, True)
print(result)

Input =  tf.Tensor(
[[[ 1. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]], shape=(5, 1, 2), dtype=float32)
Neighbour Aggregation =  tf.Tensor(
[[ 1. -1.]
 [ 0. -2.]
 [ 0. -2.]
 [ 0. -2.]
 [ 0. -1.]], shape=(5, 2), dtype=float32)
Output =  tf.Tensor(
[[0.10613449]
 [0.11824992]
 [0.11824992]
 [0.11824992]
 [0.06963718]], shape=(5, 1), dtype=float32)
Input =  tf.Tensor(
[[[ 1. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]], shape=(5, 1, 2), dtype=float32)
Neighbour Aggregation =  tf.Tensor(
[[ 1. -1.]
 [ 0. -2.]
 [ 0. -2.]
 [ 0. -2.]
 [ 0. -1.]], shape=(5, 2), dtype=float32)
Output =  tf.Tensor(
[[0.13869065]
 [0.13738433]
 [0.13738433]
 [0.13738433]
 [0.08209348]], shape=(5, 1), dtype=float32)
Input =  tf.Tensor(
[[[ 1. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]], shape=(5, 1, 2), dtype=float32)
Neighbour Aggregation =  tf.Tensor(
[[ 1. -1.]
 [ 0. -2.]
 [ 0. -2.]
 [ 0. -2.]
 [ 0. -1.]], shape=(5, 2), dtype=float32)
Output =  tf.Tensor(
